In [11]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:
from sklearn import preprocessing
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
import numpy as np
import os
import sys
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics
import itertools
#import other python scripts for further anlaysis
import classification
# Initialization of directory information:
thisDir = os.path.expanduser('~/Desktop/MSC_Alexis/analysis/')
#dataDir = thisDir + 'data/mvpa_data/tmask_all/'
dataDir = thisDir + 'data/mvpa_data/'
outDir = thisDir + 'output/subNetwork/'
#outDir = thisDir + 'output/mLmax/results/ridge/subNetwork/'
# Subjects and tasks
taskList=['mixed', 'motor','mem']
subList=['MSC01','MSC02','MSC03','MSC04','MSC05','MSC06','MSC07','MSC10']
#subList=['MSC01','MSC02','MSC04','MSC05','MSC10']
#all possible combinations of subs and tasks
subsComb=(list(itertools.permutations(subList, 2)))
tasksComb=(list(itertools.permutations(taskList, 2)))
#DS combination
DSvars=list(itertools.product(list(subsComb),list(taskList)))
##SS combination
SSvars=list(itertools.product(list(subList),list(tasksComb)))
#BS combination
BSvars=list(itertools.product(list(subsComb),list(tasksComb)))


""" run_prediction initializes what type of analysis you would like to do
and what classifier you would like to use. For now classifier options are svm:linear svm, logreg: logistic
regression, and ridge:ridge regression. Analysis is the type of analysis you wanted
to run. DS--different subject same task; SS--same subject different task;
BS--different subject different task. Each analysis will concatenate across
subjects and make a dataframe. If FW is true will collect all necessary feature weights and plot or save then
into the appropriate format. """
def classifyDS(classifier, analysis, network):
    acc_scores_per_task=[]
    tmp_df=pd.DataFrame(DSvars, columns=['sub','task'])
    dfDS=pd.DataFrame()
    dfDS[['train_sub','test_sub']]=pd.DataFrame(tmp_df['sub'].tolist())
    dfDS['task']=tmp_df['task']
    for index, row in dfDS.iterrows():
        score=model(classifier, analysis, network, train_sub=row['train_sub'], test_sub=row['test_sub'], train_task=row['task'], test_task=row['task'])
        acc_scores_per_task.append(score)
    dfDS['acc']=acc_scores_per_task
    dfDS.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/acc.csv')
    statsACC(dfDS, classifier, analysis, network)
def classifySS(classifier,analysis, network):
    acc_scores_per_task=[]
    tmp_df=pd.DataFrame(SSvars, columns=['sub','task'])
    dfSS=pd.DataFrame()
    dfSS[['train_task','test_task']]=pd.DataFrame(tmp_df['task'].tolist())
    dfSS['sub']=tmp_df['sub']
    for index, row in dfSS.iterrows():
        score=model(classifier, analysis, network, train_sub=row['sub'], test_sub=row['sub'], train_task=row['train_task'], test_task=row['test_task'])
        acc_scores_per_task.append(score)
    dfSS['acc']=acc_scores_per_task
    #save accuracy
    dfSS.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/acc.csv')
    statsACC(dfSS, classifier, analysis, network)
def classifyBS(classifier, analysis, network):
    acc_scores_per_task=[]
    tmp_df=pd.DataFrame(BSvars, columns=['sub','task'])
    dfBS=pd.DataFrame()
    dfBS[['train_task','test_task']]=pd.DataFrame(tmp_df['task'].tolist())
    dfBS[['train_sub', 'test_sub']]=pd.DataFrame(tmp_df['sub'].tolist())
    for index, row in dfBS.iterrows():
        score=model(classifier, analysis, network, train_sub=row['train_sub'], test_sub=row['test_sub'], train_task=row['train_task'], test_task=row['test_task'])
        acc_scores_per_task.append(score)
    dfBS['acc']=acc_scores_per_task
    #results.plotACC(dfBS, classifier, analysis)
    #results.statsACC(dfBS, classifier, analysis, 'fp_co')
    #results.boxACC(dfBS, classifier, analysis)
    #save accuracy
    dfBS.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/acc.csv')
    statsACC(dfBS, classifier, analysis, network)

def model(classifier, analysis, network, train_sub, test_sub, train_task, test_task):
    if classifier=='SVC':
        clf=LinearSVC(max_iter=10000, dual=False)
    elif classifier=='logReg':
        clf=LogisticRegression(solver = 'lbfgs', max_iter=10000)
    elif classifier=='ridge':
        clf=RidgeClassifier(max_iter=10000)
    else:
        print('Error: You didnt specify what classifier')
    taskFC=classification.subNets(dataDir+train_task+'/'+train_sub+'_parcel_corrmat.mat', network)
    restFC=classification.subNets(dataDir+'rest/'+train_sub+'_parcel_corrmat.mat', network)
    #if your subs are the same
    if train_sub==test_sub:
        test_taskFC=classification.subNets(dataDir+test_task+'/'+test_sub+'_parcel_corrmat.mat',network)
        ACCscores=CV_folds(clf, analysis, taskFC, restFC, test_taskFC, restFC)
    else:
        test_taskFC=classification.subNets(dataDir+test_task+'/'+test_sub+'_parcel_corrmat.mat',network)
        test_restFC=classification.subNets(dataDir+'rest/'+test_sub+'_parcel_corrmat.mat',network)
        ACCscores=CV_folds(clf, analysis, taskFC, restFC, test_taskFC, test_restFC)
    return ACCscores
#Calculate acc of cross validation within sub within task
def classifyCV(classifier, analysis, network):
    avg_CV=[]
    if classifier=='SVC':
        clf=LinearSVC(max_iter=10000, dual=False)
    elif classifier=='logReg':
        clf=LogisticRegression(solver = 'lbfgs', max_iter=10000)
    elif classifier=='ridge':
        clf=RidgeClassifier(max_iter=10000)
    else:
        print('invalid classifier')
    for task in taskList:
        acc_scores_per_task=[]
        cvTable=[]
        for sub in subList:
            taskFC=classification.subNets(dataDir+task+'/'+sub+'_parcel_corrmat.mat',network)
            restFC=classification.subNets(dataDir+'rest/'+sub+'_parcel_corrmat.mat',network)
            folds=taskFC.shape[0]
            x_train, y_train=classification.concateFC(taskFC, restFC)
            CVscores=cross_val_score(clf, x_train, y_train, cv=folds)
            mu=CVscores.mean()
            acc_scores_per_task.append(mu)
            cv_tmp_df=pd.DataFrame({sub:CVscores})
            cvTable.append(cv_tmp_df)
    #acc per fold per sub
        tmp_df=pd.DataFrame({'sub':subList, task:acc_scores_per_task}).set_index('sub')
        avg_CV.append(tmp_df)
        cvTable=pd.concat(cvTable, axis=1)
    #saving cv per folds if debugging
        cvTable.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/'+task+'_cvTable_folds.csv')
    #average acc per sub per tasks
    df=pd.concat(avg_CV, axis=1)
    df.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/acc.csv')
    statsACC(df, classifier, analysis, network)
def CV_folds(clf, analysis, taskFC, restFC, test_taskFC, test_restFC):
    loo = LeaveOneOut()
    taskSize=taskFC.shape[0]
    restSize=restFC.shape[0]
    t = np.ones(taskSize, dtype = int)
    r=np.zeros(restSize, dtype=int)
    
    if analysis=='SS':
        df=pd.DataFrame()
        acc_score=[]
        for train_index, test_index in loo.split(taskFC):
            Xtrain_rest, Xtest_rest=restFC[train_index], restFC[test_index]
            Xtrain_task=taskFC[train_index]
            ytrain_rest=r[train_index]
            ytrain_task=t[train_index]
            X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
            y_tr = np.concatenate((ytrain_task,ytrain_rest))
            #implementing standardization
            scaler = preprocessing.StandardScaler().fit(X_tr)
            scaler.transform(X_tr)
            clf.fit(X_tr,y_tr)
            tmpdf=pd.DataFrame()
            acc_scores_per_fold=[]
            for t_index, te_index in loo.split(test_taskFC):
                Xtest_task=test_taskFC[te_index]
                X_Test = np.concatenate((Xtest_task, Xtest_rest))
                y_Test = np.array([1, 0])
                #test set
                #standardization
                scaler.transform(X_Test)
                clf.predict(X_Test)
                #Get accuracy of model
                ACCscores=clf.score(X_Test,y_Test)
                acc_scores_per_fold.append(ACCscores)
            tmpdf['inner_fold']=acc_scores_per_fold
            score=tmpdf['inner_fold'].mean()
            acc_score.append(score)
        df['outer_fold']=acc_score
        total_score=df['outer_fold'].mean()
    else:
        df=pd.DataFrame()
        acc_score=[]
        #fold each training set
        for train_index, test_index in loo.split(taskFC):
            Xtrain_rest=restFC[train_index]
            Xtrain_task=taskFC[train_index]
            ytrain_rest=r[train_index]
            ytrain_task=t[train_index]
            X_tr=np.concatenate((Xtrain_task, Xtrain_rest))
            y_tr = np.concatenate((ytrain_task,ytrain_rest))
            #implement standardization
            scaler = preprocessing.StandardScaler().fit(X_tr)
            scaler.transform(X_tr)
            clf.fit(X_tr,y_tr)
            tmpdf=pd.DataFrame()
            acc_scores_per_fold=[]
            #fold each testing set
            for t_index, te_index in loo.split(test_taskFC):
                Xtest_rest=test_restFC[te_index]
                Xtest_task=test_taskFC[te_index]
                X_te=np.concatenate((Xtest_task, Xtest_rest))
                y_te=np.array([1, 0])
                #test set
                #standardization
                scaler.transform(X_te)
                clf.predict(X_te)
                #Get accuracy of model
                ACCscores=clf.score(X_te,y_te)
                acc_scores_per_fold.append(ACCscores)
            tmpdf['inner_fold']=acc_scores_per_fold
            score=tmpdf['inner_fold'].mean()
            acc_score.append(score)
        df['outer_fold']=acc_score
        total_score=df['outer_fold'].mean()

    return total_score

def statsACC(df, classifier, analysis, network):
    if analysis=='CV':
        print('cross validation stats')
        mu=df.mean()
        sd=df.std()
        stats=pd.DataFrame({'Mean':mu, 'Std':sd})
        stats.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/stats.csv', index=True)
    elif analysis=='SS':
        print('same sub stats')
        df.drop(['sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/stats.csv', index=True)
    elif analysis=='DS':
        print('diff sub stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby('task').mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby('task').std()
        stats['Std']=sd['acc']
        stats.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/stats.csv', index=True)
    elif analysis=='BS':
        print('diff sub diff task stats')
        df.drop(['train_sub', 'test_sub'], axis=1, inplace=True)
        stats=df.groupby(['test_task', 'train_task']).mean()
        stats.rename(columns={'acc':'Mean'}, inplace=True)
        sd=df.groupby(['test_task', 'train_task']).std()
        stats['Std']=sd['acc']
        stats.reset_index(inplace=True)
        stats.to_csv(outDir+network+'/results/'+classifier+'/acc/'+analysis+'/stats.csv', index=True)
    else:
        print('skipping stats')

In [8]:
networkList=['unassign',
    'default',
    'visual',
    'fp',
    'dan',
    'van',
    'salience',
    'co',
    'sm',
    'sm-lat',
    'auditory',
    'pmn',
    'pon']
#analysisList=['DS', 'SS', 'BS','CV']
#classifierList=['SVC', 'logReg','ridge']
for network in networkList:
    print('Now running for network '+network)
    classifyCV('ridge', 'CV', network)
    print("Finished CV")
    classifyDS('ridge', 'DS', network)
    print("Finished DS")
    classifySS('ridge', 'SS', network)
    print("Finished SS")
    classifyBS('ridge', 'BS', network)
    print("Finished BS")


Now running for network unassign
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network default
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network visual
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network fp
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network dan
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network van
cross validation stats
Finished CV
diff sub stats
Finished DS
same sub stats
Finished SS
diff sub diff task stats
Finished BS
Now running for network salience
cross validation stats
Finished 

In [10]:
classifyCV('ridge', 'CV', 'unassign')
print("Finished CV")
classifyDS('ridge', 'DS', 'unassign')
print("Finished DS")
classifySS('ridge', 'SS', 'unassign')
print("Finished SS")
classifyBS('ridge', 'BS', 'unassign')
print("Finished BS")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Alexis/Desktop/MSC_Alexis/analysis/output/subNetwork/unassign/CV/mixed_cvTable_folds.csv'